In [1]:
import sys
sys.path.append("../")
from tuneavideo.pipelines.pipeline_tuneavideo import TuneAVideoPipeline
from tuneavideo.pipelines.pipeline_tuneavideo_video2video import TuneAVideoVideo2VideoPipeline
from tuneavideo.pipelines import pipeline_tuneavideo_video2video
from diffusers.models.unet_2d_condition import UNet2DConditionModel

from tuneavideo.models.unet import UNet3DConditionModel
from tuneavideo.util import save_videos_grid, read_gif, save_as_gif
import torch
from einops import rearrange
import einops
import decord
from matplotlib import pyplot as plt
from PIL import Image
from moviepy import editor
import numpy as np
import torchinfo
decord.bridge.set_bridge('torch')

/mnt/disks/disk_main/anaconda3/envs/tune_a_video/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Unet2D
config = UNet3DConditionModel.load_config("../checkpoints/dedede/unet/config.json")
model = UNet2DConditionModel.from_config(config)

The config attributes {'$comment': 'delete mid_block_type'} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [4]:
timestep = 1
encoder_hidden_states = torch.randn(1, 320, 768)

torchinfo.summary(model=model, input_size=(1, 4, 64, 64), timestep=timestep, encoder_hidden_states=encoder_hidden_states, depth=100)

Layer (type:depth-idx)                                            Output Shape              Param #
UNet2DConditionModel                                              [1, 4, 64, 64]            --
├─Timesteps: 1-1                                                  [1, 320]                  --
├─TimestepEmbedding: 1-2                                          [1, 1280]                 --
│    └─Linear: 2-1                                                [1, 1280]                 410,880
│    └─SiLU: 2-2                                                  [1, 1280]                 --
│    └─Linear: 2-3                                                [1, 1280]                 1,639,680
├─Conv2d: 1-3                                                     [1, 320, 64, 64]          11,840
├─ModuleList: 1-4                                                 --                        --
│    └─CrossAttnDownBlock2D: 2-4                                  [1, 320, 32, 32]          --
│    │    └─ModuleList: 3-3  

In [2]:
# Unet3D
config = UNet3DConditionModel.load_config("../checkpoints/tav_yor_dedede/unet/config.json")
model =UNet3DConditionModel.from_config(config)

The config attributes {'$comment': 'delete mid_block_type'} were passed to UNet3DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
sample = torch.randn(1, 4, 1, 64, 64)
timestep = 1
encoder_hidden_states = torch.randn(1, 320, 768)

torchinfo.summary(model=model, input_size=(1, 4, 2, 64, 64), timestep=timestep, encoder_hidden_states=encoder_hidden_states, depth=100)
# res = model(sample, timestep, encoder_hidden_states)

Layer (type:depth-idx)                                            Output Shape              Param #
UNet3DConditionModel                                              [1, 4, 2, 64, 64]         --
├─Timesteps: 1-1                                                  [1, 320]                  --
├─TimestepEmbedding: 1-2                                          [1, 1280]                 --
│    └─Linear: 2-1                                                [1, 1280]                 410,880
│    └─SiLU: 2-2                                                  [1, 1280]                 --
│    └─Linear: 2-3                                                [1, 1280]                 1,639,680
├─InflatedConv3d: 1-3                                             [1, 320, 2, 64, 64]       11,840
├─ModuleList: 1-4                                                 --                        --
│    └─CrossAttnDownBlock3D: 2-4                                  [1, 320, 2, 32, 32]       --
│    │    └─ModuleList: 3-3  

In [14]:
res.sample.shape

torch.Size([1, 4, 1, 64, 64])